# ВКР на политфаке: how to?

## Part II: лемматизировали, лемматизировали, да не вылемматизировали...

В этой части мы выполняем формальную обработку текстов 

*Никита Смирнов, Анна Иванешкина, БПТ-181*

**0. Подгрузка всех необходимых пакетов**

In [2]:
import numpy as np
import pandas as pd 
import nltk

from nltk.tokenize import TweetTokenizer
from pymystem3 import Mystem

In [3]:
vkr = pd.read_csv(r'C:\Users\dramm\Desktop\vkr.csv', index_col='Unnamed: 0')
vkr.head()

,papername,annotlink,text,student,year,supervisor,superlink,program,mark,annot
0,"""Абсурдные"" электоральные акторы: происхождени...",https://www.hse.ru/edu/vkr/364642404,1,Коротеева Полина Петровна,2020,Каспэ Святослав Игоревич,www.hse.ru/org/persons/67280,Политология,8.0,"Данная работа посвящена изучению того, какие ф..."
1,Взаимосвязь между стабильностью политического ...,https://www.hse.ru/edu/vkr/366243110,0,Минина Анастасия Алексеевна,2020,Розенберг Дина Яновна,www.hse.ru/org/persons/64235685,Политология,NaN,Работа посвящена изучению террористической акт...
2,Взаимосвязь между электронным правительством и...,https://www.hse.ru/edu/vkr/366241655,0,Науширванов Тимур Олегович,2020,Розенберг Дина Яновна,www.hse.ru/org/persons/64235685,Политология,NaN,Исследование посвящено изучению взаимосвязей м...
3,Взаимосвязь социально-политической напряженнос...,https://www.hse.ru/edu/vkr/366244578,1,Локаева Екатерина Андреевна,2020,Туровский Ростислав Феликсович,www.hse.ru/org/persons/67524,Политология,8.0,Данное исследование фокусируется на выявлении ...
4,Взаимосвязь уровня коррупции и гендерного нера...,https://www.hse.ru/edu/vkr/366242960,0,Макарова Анастасия Вадимовна,2020,Сальникова Дарья Вячеславовна,www.hse.ru/staff/salnikova,Политология,NaN,В настоящей работе целью исследования является...


**1. Стоп-слова и токенизация**

In [4]:
nltk.download("stopwords")

from nltk.corpus import stopwords
stop_words = stopwords.words('russian')

np.array(stop_words)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dramm\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


array(['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как',
       'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у',
       'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот',
       'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда',
       'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть',
       'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь',
       'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут',
       'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем',
       'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже',
       'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того',
       'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом',
       'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были',
       'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец',
       'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот',
 

In [5]:
tokenizer = TweetTokenizer()

def preprocess(text : str, tokenizer):
    return ' '.join(
        [word for word in tokenizer.tokenize(text.lower()) if word not in stop_words]
        )

In [6]:
vkr['annottoken'] = vkr['annot'].astype(str).apply(preprocess, args=(tokenizer, ))
vkr.head()

,papername,annotlink,text,student,year,supervisor,superlink,program,mark,annot,annottoken
0,"""Абсурдные"" электоральные акторы: происхождени...",https://www.hse.ru/edu/vkr/364642404,1,Коротеева Полина Петровна,2020,Каспэ Святослав Игоревич,www.hse.ru/org/persons/67280,Политология,8.0,"Данная работа посвящена изучению того, какие ф...","данная работа посвящена изучению , какие факто..."
1,Взаимосвязь между стабильностью политического ...,https://www.hse.ru/edu/vkr/366243110,0,Минина Анастасия Алексеевна,2020,Розенберг Дина Яновна,www.hse.ru/org/persons/64235685,Политология,NaN,Работа посвящена изучению террористической акт...,работа посвящена изучению террористической акт...
2,Взаимосвязь между электронным правительством и...,https://www.hse.ru/edu/vkr/366241655,0,Науширванов Тимур Олегович,2020,Розенберг Дина Яновна,www.hse.ru/org/persons/64235685,Политология,NaN,Исследование посвящено изучению взаимосвязей м...,исследование посвящено изучению взаимосвязей э...
3,Взаимосвязь социально-политической напряженнос...,https://www.hse.ru/edu/vkr/366244578,1,Локаева Екатерина Андреевна,2020,Туровский Ростислав Феликсович,www.hse.ru/org/persons/67524,Политология,8.0,Данное исследование фокусируется на выявлении ...,данное исследование фокусируется выявлении пол...
4,Взаимосвязь уровня коррупции и гендерного нера...,https://www.hse.ru/edu/vkr/366242960,0,Макарова Анастасия Вадимовна,2020,Сальникова Дарья Вячеславовна,www.hse.ru/staff/salnikova,Политология,NaN,В настоящей работе целью исследования является...,настоящей работе целью исследования является о...


**2. Лемматизация**

In [7]:
m = Mystem()

def lemmatize(text):
    lemmas = m.lemmatize(text)
    return (''.join(lemmas))

vkr['lemmannot'] = vkr['annottoken'].apply(lemmatize)

In [8]:
vkr.head()

,papername,annotlink,text,student,year,supervisor,superlink,program,mark,annot,annottoken,lemmannot
0,"""Абсурдные"" электоральные акторы: происхождени...",https://www.hse.ru/edu/vkr/364642404,1,Коротеева Полина Петровна,2020,Каспэ Святослав Игоревич,www.hse.ru/org/persons/67280,Политология,8.0,"Данная работа посвящена изучению того, какие ф...","данная работа посвящена изучению , какие факто...","данный работа посвящать изучение , какой факто..."
1,Взаимосвязь между стабильностью политического ...,https://www.hse.ru/edu/vkr/366243110,0,Минина Анастасия Алексеевна,2020,Розенберг Дина Яновна,www.hse.ru/org/persons/64235685,Политология,NaN,Работа посвящена изучению террористической акт...,работа посвящена изучению террористической акт...,работа посвящать изучение террористический акт...
2,Взаимосвязь между электронным правительством и...,https://www.hse.ru/edu/vkr/366241655,0,Науширванов Тимур Олегович,2020,Розенберг Дина Яновна,www.hse.ru/org/persons/64235685,Политология,NaN,Исследование посвящено изучению взаимосвязей м...,исследование посвящено изучению взаимосвязей э...,исследование посвящать изучение взаимосвязь эл...
3,Взаимосвязь социально-политической напряженнос...,https://www.hse.ru/edu/vkr/366244578,1,Локаева Екатерина Андреевна,2020,Туровский Ростислав Феликсович,www.hse.ru/org/persons/67524,Политология,8.0,Данное исследование фокусируется на выявлении ...,данное исследование фокусируется выявлении пол...,данный исследование фокусироваться выявление п...
4,Взаимосвязь уровня коррупции и гендерного нера...,https://www.hse.ru/edu/vkr/366242960,0,Макарова Анастасия Вадимовна,2020,Сальникова Дарья Вячеславовна,www.hse.ru/staff/salnikova,Политология,NaN,В настоящей работе целью исследования является...,настоящей работе целью исследования является о...,настоящий работа цель исследование являться оп...


Замечательно! Теперь сохраним датафрейм в CSV, чтобы работать с ним дальше 

In [9]:
vkr.to_csv(r'C:\Users\dramm\Desktop\vkr_lemm.csv')